# StockChart.ipynb
-----------
## Yahoo Financeからシンプレクスの1年間の日次ヒストリーを取得、Yahooスタイルの出来高と株価のチャートを描くNotebookのプログラムを書いて。

以下に、yfinance でシンプレクス（4373.T）の過去 1 年の日次データを取得し、Yahooファイナンス風スタイルで株価＋出来高チャートを描く Jupyter Notebook プログラムを示します。

ポイント

- yfinance から 1 年分の日次データを取得
- mplfinance を使って Yahoo スタイルのローソク足＋出来高を描画
- Notebook でそのまま動く

In [ ]:
# ========= Install (if needed) =========
# !pip install yfinance mplfinance

import yfinance as yf
import mplfinance as mpf
import pandas as pd

# ------------------------------------------------------------
# 1. データ取得（シンプレクス：4373.T）
# ------------------------------------------------------------
ticker = "4373.T"
stock = yf.Ticker(ticker)

# 1年間の日次データ
df = stock.history(period="1y", interval="1d", auto_adjust=False, actions=False)

# 欠損除去
df = df.dropna()

print(df.head())

# ------------------------------------------------------------
# 2. Yahooスタイルでチャート描画
# ------------------------------------------------------------
mpf.plot(
    df,
    type="candle",            # ローソク足
    mav=(5, 25),             # 移動平均（必要なければ削除可能）
    volume=True,             # 出来高
    style="yahoo",           # Yahooファイナンス風
    title="Simplex Holdings (4373.T) — 1 Year Price & Volume (Yahoo Style)",
    figratio=(16, 9),
    figscale=1.2
)


📊 出力されるチャート

- 上：ローソク足（Open/High/Low/Close）
- 中：移動平均（5日・25日）
- 下：出来高 (Volume)

Yahoo Finance と同じ見た目のスタイル

🔧 オプション（必要なら追加できます）

- ボリンジャーバンド
- MACD / RSI
- インタラクティブ（Plotly）版
- スライダーで期間変更
- 複数銘柄比較（パフォーマンス指数化）

必要なら、「Plotly 版の Yahoo スタイル」 も作ります。

--------
## スライダーで期間変更して

了解です！
Yahooスタイルのローソク足チャート（mplfinance）に、スライダーで表示期間を変更できる機能を追加した Notebook 用コードをお渡しします。

In [ ]:
# ========= Install (if needed) =========
# !pip install yfinance mplfinance ipywidgets

import yfinance as yf
import mplfinance as mpf
import pandas as pd
from ipywidgets import interact, IntSlider

# ------------------------------------------------------------
# 1. データ取得（シンプレクス：4373.T）
# ------------------------------------------------------------
ticker = "4373.T"
stock = yf.Ticker(ticker)

df = stock.history(period="1y", interval="1d", auto_adjust=False, actions=False)
df = df.dropna()

# ------------------------------------------------------------
# 2. 移動平均 (MA5, MA25) 追加
# ------------------------------------------------------------
df["MA5"] = df["Close"].rolling(5).mean()
df["MA25"] = df["Close"].rolling(25).mean()

# ------------------------------------------------------------
# 3. スライダーで期間変更してチャートを描画
# ------------------------------------------------------------
def plot_with_slider(days):

    # 最新から days 日分を抽出
    data = df.tail(days)

    mpf.plot(
        data,
        type="candle",
        mav=(5, 25),            # MA5, MA25
        volume=True,            # 出来高
        style="yahoo",          # Yahooスタイル
        title=f"4373.T — Last {days} Days (Yahoo Style)",
        figratio=(16, 9),
        figscale=1.2
    )

# ------------------------------------------------------------
# 4. インタラクティブスライダー
# ------------------------------------------------------------
interact(
    plot_with_slider,
    days=IntSlider(
        min=30,
        max=len(df),
        step=5,
        value=120,
        description="表示日数"
    )
)


🎛 スライダーの動き

- 30日〜過去1年（最大365日前後） まで自由に変更
- 右に動かすほど期間が長くなる
- Yahooスタイル（ローソク＋MA5/MA25＋出来高）のまま動的更新

🔧 カスタマイズ例（希望あれば組み込みます）

- スライダーを「開始日・終了日」の 2 つにする
- 日付カレンダーウィジェットに対応
- Plotly 版でズーム＋スライダー併用
- ボリンジャーバンド追加
- MACD / RSI 追加

必要なら「Plotly 版（ローソク足 + スライダー + ズーム）」も作りますので言ってください！

---------------
## ボリンジャーバンドを追加して

了解です！
Yahooスタイルのローソク足チャート（mplfinance）に、スライダーで期間変更＋ボリンジャーバンド（20日・±2σ）を追加した完全コードをお渡しします。

🎯 機能一覧（すべて含む）

✔ yfinance から 4373.T の 1年日次データ取得
✔ Yahooスタイルのローソク足
✔ MA5 / MA25
✔ ボリンジャーバンド（20日・±2σ）
✔ 出来高
✔ スライダーで期間可変（30日〜1年）

In [ ]:
# ========= Install (if needed) =========
# !pip install yfinance mplfinance ipywidgets

import yfinance as yf
import mplfinance as mpf
import pandas as pd
from ipywidgets import interact, IntSlider

# ------------------------------------------------------------
# 1. データ取得（シンプレクス：4373.T）
# ------------------------------------------------------------
ticker = "4373.T"
stock = yf.Ticker(ticker)

df = stock.history(period="1y", interval="1d", auto_adjust=False, actions=False)
df = df.dropna()

# ------------------------------------------------------------
# 2. 移動平均 (MA5, MA25)
# ------------------------------------------------------------
df["MA5"] = df["Close"].rolling(5).mean()
df["MA25"] = df["Close"].rolling(25).mean()

# ------------------------------------------------------------
# 3. ボリンジャーバンド（20日）
# ------------------------------------------------------------
window = 20
df["BB_MID"] = df["Close"].rolling(window).mean()
df["BB_STD"] = df["Close"].rolling(window).std()
df["BB_UPPER"] = df["BB_MID"] + 2 * df["BB_STD"]
df["BB_LOWER"] = df["BB_MID"] - 2 * df["BB_STD"]

# ------------------------------------------------------------
# 4. mpf 用にプロット追加設定
# ------------------------------------------------------------
# mpf.make_addplot() でローソク足にラインを重ねる
add_plots = [
    mpf.make_addplot(df["BB_MID"], color="blue", width=1),
    mpf.make_addplot(df["BB_UPPER"], color="green", width=1),
    mpf.make_addplot(df["BB_LOWER"], color="red", width=1),
]

# ------------------------------------------------------------
# 5. スライダーで期間変更しながらプロット
# ------------------------------------------------------------
def plot_with_slider(days):

    data = df.tail(days)

    mpf.plot(
        data,
        type="candle",
        mav=(5, 25),              # MA5, MA25
        volume=True,
        style="yahoo",
        addplot=[
            mpf.make_addplot(data["BB_MID"], color="blue", width=1),
            mpf.make_addplot(data["BB_UPPER"], color="green", width=1),
            mpf.make_addplot(data["BB_LOWER"], color="red", width=1)
        ],
        title=f"4373.T — Last {days} Days (Yahoo Style + Bollinger Bands)",
        figratio=(16, 9),
        figscale=1.2
    )

# ------------------------------------------------------------
# 6. インタラクティブスライダー
# ------------------------------------------------------------
interact(
    plot_with_slider,
    days=IntSlider(
        min=30,
        max=len(df),
        step=5,
        value=120,
        description="表示日数"
    )
)


📊 出力されるチャート

ローソク足（Yahoo Style）
- MA5（黄）
- MA25（紫）
- ボリンジャーバンド（青＝中心線、緑＝上限、赤＝下限）
- 出来高
- スライダーで期間調整

🔧 カスタマイズ可能

- BB を ±1σ / ±3σ に変更
- BB を帯表示（塗りつぶし）に変更
- RSI / MACD を別パネルに追加
- Plotly 版のボリンジャーバンド
- 移動平均を EMA に切り替え

次に追加したい機能はありますか？

---------------
## グラフの凡例を追加して

了解です！
mplfinance は 通常の Matplotlib のように自動で凡例（legend）を出してくれないため、
カスタムで Axes オブジェクトに凡例を追加する必要があります。

以下のコードは：

- Yahooスタイルローソク足
- MA5 / MA25
- ボリンジャーバンド（中心線・上限・下限）
- 出来高
- スライダーで期間変更
- カスタム凡例追加（Matplotlib の legend を自動生成）

をすべて含んだ、完全版 Notebook コードです。

In [ ]:
# ========= Install (if needed) =========
# !pip install yfinance mplfinance ipywidgets

import yfinance as yf
import mplfinance as mpf
import pandas as pd
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 1. データ取得（シンプレクス：4373.T）
# ------------------------------------------------------------
ticker = "4373.T"
stock = yf.Ticker(ticker)

df = stock.history(period="1y", interval="1d", auto_adjust=False, actions=False)
df = df.dropna()

# ------------------------------------------------------------
# 2. 移動平均 (MA5, MA25)
# ------------------------------------------------------------
df["MA5"] = df["Close"].rolling(5).mean()
df["MA25"] = df["Close"].rolling(25).mean()

# ------------------------------------------------------------
# 3. ボリンジャーバンド（20日）
# ------------------------------------------------------------
window = 20
df["BB_MID"] = df["Close"].rolling(window).mean()
df["BB_STD"] = df["Close"].rolling(window).std()
df["BB_UPPER"] = df["BB_MID"] + 2 * df["BB_STD"]
df["BB_LOWER"] = df["BB_MID"] - 2 * df["BB_STD"]

# ------------------------------------------------------------
# 4. スライダー付きプロット関数（凡例付き）
# ------------------------------------------------------------
def plot_with_slider(days):

    data = df.tail(days)

    # mpf.plot の戻り値に fig, axes を受け取る
    fig, axes = mpf.plot(
        data,
        type="candle",
        mav=(5, 25),
        volume=True,
        style="yahoo",
        addplot=[
            mpf.make_addplot(data["BB_MID"], color="blue", width=1),
            mpf.make_addplot(data["BB_UPPER"], color="green", width=1),
            mpf.make_addplot(data["BB_LOWER"], color="red", width=1)
        ],
        title=f"4373.T — Last {days} Days (Yahoo Style + Bollinger Bands)",
        figratio=(16, 9),
        figscale=1.2,
        returnfig=True
    )

    ax_price = axes[0]      # 価格チャートの Axes
    ax_volume = axes[2]     # 出来高の Axes

    # --------------------------------------------------------
    # 5. 凡例の追加（ローソク足上に追加）
    # --------------------------------------------------------
    # ラインをまとめて凡例生成
    lines = [
        ax_price.lines[0],   # MA5
        ax_price.lines[1],   # MA25
        ax_price.lines[2],   # BB MID
        ax_price.lines[3],   # BB Upper
        ax_price.lines[4],   # BB Lower
    ]

    labels = [
        "MA5",
        "MA25",
        "BB Mid",
        "BB Upper",
        "BB Lower"
    ]

    ax_price.legend(lines, labels, loc="upper left")

    plt.show()

# ------------------------------------------------------------
# 6. インタラクティブスライダー
# ------------------------------------------------------------
interact(
    plot_with_slider,
    days=IntSlider(
        min=30,
        max=len(df),
        step=5,
        value=120,
        description="表示日数"
    )
)


📌 凡例はこう動きます

- MA5（黄）
- MA25（紫）
- ボリンジャーバンド中心（青）
- 上限（緑）
- 下限（赤）

が ローソク足チャートの左上に表示されます。

🔧 さらに改善できます

必要なら：

- 凡例の位置を移動（右上・外側など）
- ボリンジャーバンドの背景を塗りつぶす（帯状表示）
- 出来高にも移動平均線追加
- Plotly 版インタラクティブ BB チャート

など追加できます。

続けて何を加えますか？